# SDC SocialMedia Assist RAG + Chat

## Import Libs 

In [1]:
from langchain.chains import RetrievalQA
from langchain.globals import set_debug
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_community import BigQueryVectorStore, VertexFSVectorStore,GCSFileLoader
from langchain_google_vertexai import VertexAI, VertexAIEmbeddings

from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from google.cloud import storage
import io
from dotenv import dotenv_values

config = dotenv_values("config/config.env")

## Loading Params

In [2]:
PROJECT_ID = config["PROJECT_ID"]
LOCATION = config["LOCATION"]

# CloudStorage
BUCKET_NAME = config["BUCKET_NAME"]
BLOB_NAME = config["BLOB_NAME"]

# BigQuery
DATASET_ID = config["DATASET_ID"]
TABLE_ID = config["TABLE_ID"]

# Models
MODEL_NAME = config["MODEL_NAME"]
EMBEDDING_MODEL = config["EMBEDDING_MODEL"]

## RAG

In [3]:
def build_embedding_model(embedding_model, project_id):
    
    embedding_model = VertexAIEmbeddings(
        model_name=embedding_model, project=project_id
    )
    
    return embedding_model

In [4]:
def initialize_bqvector(project_id, location, dataset_id, table_id, embedding_model):
    bq_store = BigQueryVectorStore(
        project_id=project_id,
        location=location,
        dataset_name=dataset_id,
        table_name=table_id,
        embedding=embedding_model,
    )
    return bq_store

In [5]:
def build_system_prompt():
    
    # Define your custom prompt
    custom_prompt = PromptTemplate(
        input_variables=["context", "question", "chat_history"],
        template="""
        Du bist ein hilfsbereiter KI-Assistent für unser Unternehmen, spezialisiert auf Social-Media-Strategien und die Einarbeitung neuer Teammitglieder. 
        Nutze das vorhandene Wissen aus dem Kontext, um präzise und nützliche Antworten zu liefern.

        Gesprächsverlauf:
        {chat_history}

        Kontext:
        {context}

        Frage:
        {question}

        Bitte beachte bei deiner Antwort:
        - Gehe spezifisch auf die Bedürfnisse des Nutzers ein.
        - Nur auf Nachfrage! Biete praktische Tipps für Social-Media-Posts im Unternehmenskontext.
        - Unterstütze neue Kollegen mit klaren Anweisungen und Ressourcen.
        - Verwende eine positive und motivierende Sprache.
        - Halte dich kurze und beantworte im ersten schritt erst mal nur die Nutzer-Frage.
        - Gebe mir Antwort wenn passend als Aufzählung auf.

        Antwort:
        """
        )
    return custom_prompt

def initialize_llm_model(model_name):
    # Initialize the LLM
    llm = VertexAI(model_name=model_name)
    
    return llm

def create_memory_buffer():
    # Set up the conversation memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        input_key="question",
        output_key="answer",
        return_messages=True,
        verbose=True
    )
    
    return memory

def convo_chain(llm, langchain_retriever, memory, custom_prompt):
    # Set up the ConversationalRetrievalChain with the custom prompt
    conversational_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=langchain_retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": custom_prompt},
        # verbose=True
    )
    
    return conversational_chain

# Function to interact with the agent
def chat_with_agent(conversational_chain):
    print("Start chatting with the agent (type 'exit' to stop):")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            break
        response = conversational_chain.invoke({"question": user_input})
        print(f"\nAssistant: {response['answer']}")

In [7]:
print("01 - Initialize embedding model")
embedding_model = build_embedding_model(EMBEDDING_MODEL, PROJECT_ID)

print("02 - Initialize BQVector Storage")
bq_store = initialize_bqvector(PROJECT_ID,LOCATION,DATASET_ID,TABLE_ID,embedding_model)

print("03 - Langchain retriever")
langchain_retriever = bq_store.as_retriever()

print("04 - Build System Prompt")
custom_prompt = build_system_prompt()

print("05 - Initialize LLM model")
llm = initialize_llm_model(MODEL_NAME)

print("06 - Initialize meory")
memory = create_memory_buffer()

print("07 - Conversational chain")
conversational_chain = convo_chain(llm, langchain_retriever, memory, custom_prompt)

# Start the chat
chat_with_agent(conversational_chain)

01 - Initialize embedding model
02 - Initialize BQVector Storage
BigQuery table sdc-gen-ai.sdc_marketing.sdc_instagram_guideline initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=sdc-gen-ai&ws=!1m5!1m4!4m3!1ssdc-gen-ai!2ssdc_marketing!3ssdc_instagram_guideline


/var/tmp/ipykernel_89445/3798123653.py:40: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


03 - Langchain retriever
04 - Build System Prompt
05 - Initialize LLM model
06 - Initialize meory
07 - Conversational chain
Start chatting with the agent (type 'exit' to stop):



You:  was sind die Technische Hintergünde für einen beitrag?




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        Du bist ein hilfsbereiter KI-Assistent für unser Unternehmen, spezialisiert auf Social-Media-Strategien und die Einarbeitung neuer Teammitglieder. 
        Nutze das vorhandene Wissen aus dem Kontext, um präzise und nützliche Antworten zu liefern.

        Gesprächsverlauf:
        

        Kontext:
        Skizze: Leitfaden Instagram Postings

1. Welchen Zweck hat das Posting? Überleg dir klare Ziele für das Posting 2. Welches Posting-Format (Reel, Story, Beitrag) wählst du? Habe das zugehörige Video-Format im Hinterkopf (s.u. “Technische Hintergründe”) 3. Werte und Ziele - Check! Überprüfe ob dein Vorhaben im Einklang steht: 4. 5. 6. 7. 8. 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23.

4. Wie bindest du die Community mit ein? Umfragen, Countdown-Sticker, zum Kommentieren auffordern (z.B. heute haben wir was zu python gelernt….was ist eure liebste Programmie


You:  exit
